# FEM mode solver 

To understand the propagation in a waveguide, it's often helpful to look at how the effective refractive indices of the modes change when adjusting the width of the waveguide.
As the modes continuously change their refractive index, we can track them through their evolution.
We gray the area which would indicate a effective refractive index below the refractive index of the underlying layer (commonly referred to as box), as such modes would not be guided.
The refractive index of the box called "cutoff", as it defines the effective refractive index level under which modes stop being guided.

In [ ]:
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
from femwell.maxwell.waveguide import compute_modes
from femwell.mesh import mesh_from_OrderedDict
from shapely.geometry import Polygon, box
from shapely.ops import clip_by_rect
from skfem import Basis, ElementTriP0
from skfem.io.meshio import from_meshio
from tqdm import tqdm

In [ ]:
wavelength = 1.55  # wavelength of the light
ncore = 1.9963  # refractive index of the core (SiN)
nclad = 1.444  # refractive index of the cladding (SiO2)
num_modes = 8
wg_thickness = 0.33
widths = np.linspace(0.5, 3.5, 100)

all_neffs = np.zeros((widths.shape[0], num_modes))
all_te_fracs = np.zeros((widths.shape[0], num_modes))
for i, width in enumerate(tqdm(widths)):
    core = box(0, 0, width, wg_thickness)
    polygons = OrderedDict(
        core=core,
        box=clip_by_rect(core.buffer(1.0, resolution=4), -np.inf, -np.inf, np.inf, 0),
        clad=clip_by_rect(core.buffer(1.0, resolution=4), -np.inf, 0, np.inf, np.inf),
    )

    resolutions = {"core": {"resolution": 0.1, "distance": 1}}

    mesh = from_meshio(
        mesh_from_OrderedDict(polygons, resolutions, default_resolution_max=0.6)
    )

    basis0 = Basis(mesh, ElementTriP0())
    epsilon = basis0.zeros(dtype=complex)
    for subdomain, n in {"core": ncore, "box": nclad, "clad": nclad}.items():
        epsilon[basis0.get_dofs(elements=subdomain)] = n**2

    modes = compute_modes(basis0, epsilon, wavelength=wavelength, num_modes=num_modes)
    all_neffs[i] = np.real([mode.n_eff for mode in modes])
    all_te_fracs[i, :] = [mode.te_fraction for mode in modes]

In [ ]:
all_neffs = np.real(all_neffs)
plt.xlabel("Width of waveguide / µm")
plt.ylabel("Effective refractive index")
plt.fill_between(widths, nclad, alpha=0.5, color="gray")
plt.ylim(1.36, np.max(all_neffs) + 0.1 * (np.max(all_neffs) - 1.444))
for lams, te_fracs in zip(all_neffs.T, all_te_fracs.T):
    plt.plot(widths, lams)
    plt.scatter(widths, lams, c=te_fracs, cmap="cool")
plt.colorbar().set_label("TE fraction")
plt.title("Effective refractive index vs width of waveguide of SiN")
plt.show()

The graph shows a TE mode emerging (getting a greater effective refractive index than the box) at a width of ~750nm.
The second mode emerges at a width of ~1700nm.
Thus, the waveguide is a single mode waveguide with a width within the range from ~750nm to 1700nm.

The second mode is at the width at which it emerges a TM-mode, but shows for slightly wider waveguides an anti-crossing with the emerging TE-mode.
After this anti-crossing, the mode with the second highest effective refracteive index is TE10-mode, while the TM00-mode keeps existing in the system with an effective refractive index slightly above the cutoff.
At a  width of ~2600nm, a third TE-mode starts being guided.

In [ ]:
wavelength = 1.55  # wavelength of the light
ncore = 3.47  # refractive index of the core (Si)
nclad = 1.444  # refractive index of the cladding (SiO2)
num_modes = 6
widths = np.array([0.5])
wg_thickness = 0.22
sidewall_angle = 0  # sidewall angle of the waveguide (in degrees)
slab_thickness = 0
default_resolution_max = 0.08

core_resolution = 0.02
core_distance = 1
buffer_size = 1.0


all_neffs = np.zeros((widths.shape[0], num_modes))
all_te_fracs = np.zeros((widths.shape[0], num_modes))
for i, width in enumerate(tqdm(widths)):
    top_width = width
    bottom_width = 2 * np.tan(np.deg2rad(sidewall_angle)) + width
    total_slab_width = 4 + bottom_width

    if slab_thickness > 0:
        core = Polygon(
            [
                (-total_slab_width / 2, 0),
                (+total_slab_width / 2, 0),
                (+total_slab_width / 2, slab_thickness),
                (+bottom_width / 2, slab_thickness),
                (+top_width / 2, wg_thickness),
                (-top_width / 2, wg_thickness),
                (-bottom_width / 2, slab_thickness),
                (-total_slab_width / 2, slab_thickness),
                (-total_slab_width / 2, 0),
            ]
        )
    else:
        core = Polygon(
            [
                (-bottom_width / 2, 0),
                (+bottom_width / 2, 0),
                (+top_width / 2, wg_thickness),
                (-top_width / 2, wg_thickness),
                (-bottom_width / 2, 0),
            ]
        )

    polygons = OrderedDict(
        core=core,
        box=clip_by_rect(
            core.buffer(buffer_size, resolution=1), -np.inf, -np.inf, np.inf, 0
        ),
        clad=clip_by_rect(
            core.buffer(buffer_size, resolution=1), -np.inf, 0, np.inf, np.inf
        ),
    )

    resolutions = {"core": {"resolution": core_resolution, "distance": core_distance}}

    mesh = from_meshio(
        mesh_from_OrderedDict(
            polygons, resolutions, default_resolution_max=default_resolution_max
        )
    )

    resolutions = {"core": {"resolution": core_resolution, "distance": core_distance}}
    mesh = from_meshio(
        mesh_from_OrderedDict(
            polygons, resolutions, default_resolution_max=default_resolution_max
        )
    )

    basis0 = Basis(mesh, ElementTriP0())
    epsilon = basis0.zeros(dtype=complex)
    for subdomain, n in {"core": ncore, "box": nclad, "clad": nclad}.items():
        epsilon[basis0.get_dofs(elements=subdomain)] = n**2

    modes = compute_modes(basis0, epsilon, wavelength=wavelength, num_modes=num_modes)
    all_neffs[i] = np.real([mode.n_eff for mode in modes])
    all_te_fracs[i, :] = [mode.te_fraction for mode in modes]

In [ ]:
modes[0].show("E", part="real")

In [ ]:
modes[0].plot_component("E", component="x", part="real", colorbar=True)

In [ ]:
wavelength = 1.55  # wavelength of the light
ncore = 3.47  # refractive index of the core (Si)
nclad = 1.444  # refractive index of the cladding (SiO2)
num_modes = 6
widths = np.linspace(0.2, 1.0, 100)  # width of the waveguide
wg_thickness = 0.22
sidewall_angle = 0  # sidewall angle of the waveguide (in degrees)
slab_thickness = 0
default_resolution_max = 0.6

core_resolution = 0.08
core_distance = 1
buffer_size = 1.0


all_neffs = np.zeros((widths.shape[0], num_modes))
all_te_fracs = np.zeros((widths.shape[0], num_modes))
for i, width in enumerate(tqdm(widths)):
    top_width = width
    bottom_width = 2 * np.tan(np.deg2rad(sidewall_angle)) + width
    total_slab_width = 4 + bottom_width

    if slab_thickness > 0:
        core = Polygon(
            [
                (-total_slab_width / 2, 0),
                (+total_slab_width / 2, 0),
                (+total_slab_width / 2, slab_thickness),
                (+bottom_width / 2, slab_thickness),
                (+top_width / 2, wg_thickness),
                (-top_width / 2, wg_thickness),
                (-bottom_width / 2, slab_thickness),
                (-total_slab_width / 2, slab_thickness),
                (-total_slab_width / 2, 0),
            ]
        )
    else:
        core = Polygon(
            [
                (-bottom_width / 2, 0),
                (+bottom_width / 2, 0),
                (+top_width / 2, wg_thickness),
                (-top_width / 2, wg_thickness),
                (-bottom_width / 2, 0),
            ]
        )

    polygons = OrderedDict(
        core=core,
        box=clip_by_rect(
            core.buffer(buffer_size, resolution=4), -np.inf, -np.inf, np.inf, 0
        ),
        clad=clip_by_rect(
            core.buffer(buffer_size, resolution=4), -np.inf, 0, np.inf, np.inf
        ),
    )

    resolutions = {"core": {"resolution": core_resolution, "distance": core_distance}}

    mesh = from_meshio(
        mesh_from_OrderedDict(
            polygons, resolutions, default_resolution_max=default_resolution_max
        )
    )

    resolutions = {"core": {"resolution": core_resolution, "distance": core_distance}}
    mesh = from_meshio(
        mesh_from_OrderedDict(
            polygons, resolutions, default_resolution_max=default_resolution_max
        )
    )

    basis0 = Basis(mesh, ElementTriP0())
    epsilon = basis0.zeros(dtype=complex)
    for subdomain, n in {"core": ncore, "box": nclad, "clad": nclad}.items():
        epsilon[basis0.get_dofs(elements=subdomain)] = n**2

    modes = compute_modes(basis0, epsilon, wavelength=wavelength, num_modes=num_modes)
    all_neffs[i] = np.real([mode.n_eff for mode in modes])
    all_te_fracs[i, :] = [mode.te_fraction for mode in modes]

In [ ]:
all_neffs = np.real(all_neffs)
plt.title("Effective refractive index vs width of waveguide of Si")
plt.xlabel("Width of waveguide / µm")
plt.ylabel("Effective refractive index")
plt.fill_between(widths, nclad, alpha=0.5, color="gray")
plt.ylim(1.36, np.max(all_neffs) + 0.1 * (np.max(all_neffs) - 1.444))
for lams, te_fracs in zip(all_neffs.T, all_te_fracs.T):
    plt.plot(widths, lams)
    plt.scatter(widths, lams, c=te_fracs, cmap="cool")
plt.colorbar().set_label("TE fraction")
plt.show()

The second order TE mode (pink) starts to be guided around 0.45 um, which is the single mode condition for these types of waveguides.